In [2]:

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import os
# import k-means from clustering stage
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


print('Libraries imported.')

Libraries imported.


In [3]:

wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp = requests.get(wiki_url).text

In [4]:
soup = BeautifulSoup(resp, 'xml')#Beautiful Soup to Parse the url page

In [5]:
table=soup.find('table')

In [6]:

column_names=['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

In [7]:

# extracting information from the table
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [8]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
# remove rows where Borough is 'Not assigned'
df=df[df['Borough']!='Not assigned']

In [10]:
# assign Neighbourhood=Borough where Neighbourhood is 'Not assigned'
df[df['Neighborhood']=='Not assigned']=df['Borough']

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:

# group multiple Neighbourhood under one Postcode
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [13]:
# join the newly constructed joined data frame
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [14]:
# drop the Neighbourhood column
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [15]:

# drop duplicates from the data frame
df_merge.drop_duplicates(inplace=True)

In [16]:
# rename Neighbourhood_joined back to Neighbourhood
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [17]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,Queen's Park,Queen's Park,Queen's Park


In [18]:
df_merge.shape

(103, 3)

In [18]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [19]:
# takes a long time to get the data
#get_geocode('M3A')

# so we will use the csv sheet provided by Coursera as an alternative
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, df_merge, on='Postalcode')

In [22]:

geo_merged.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [23]:
# correcting the sequence of data
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [25]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
# work with only boroughs that contain the word Toronto 
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]

In [27]:
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [24]:
CLIENT_ID = os.environ['RY52LT0GGUDX1GO3M0FEK50YHIF2V5HJ0PZHEAHQOSRASI1S'] # your Foursquare ID
CLIENT_SECRET = os.environ['TF4YHA5TNKXKQWIEHLGPYYSYVENMAKBMNNCU4MK4SR3G2ZIS'] # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

KeyError: 'RY52LT0GGUDX1GO3M0FEK50YHIF2V5HJ0PZHEAHQOSRASI1S'

In [25]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                )

NameError: name 'toronto_data' is not defined

In [28]:
toronto_venues.head()

NameError: name 'toronto_venues' is not defined

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [27]:
toronto_venues.groupby('Neighborhood').count()

NameError: name 'toronto_venues' is not defined

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) # seems like it already contains Neighborhood column as number
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'].to_list() )
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()